<a href="https://colab.research.google.com/github/alfredqbit/NU-DDS-8515/blob/main/sepulvedaADDS-8515-7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 7: Analyze Ecological Data using Multivariate Regression and CCA

Jupyter Python notebook:

 - Downloads the dune and dune.env data from public R-universe/CRAN mirrors or GitHub-like sources.

 - Computes richness and Shannon diversity.

 - Builds multivariate regression, ridge, and multitask lasso pipelines.

 - Runs CCA using scikit-bio.

 - Exports all key plots as PNGs into a figures/ subdirectory with the filenames referenced in the LaTeX.

In [ ]:
import os
from pathlib import Path

import io # Added by the user
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, RidgeCV, MultiTaskLassoCV
from sklearn.metrics import r2_score, mean_squared_error
# Perform a clean and ordered installation to resolve deep dependency conflicts
#!pip uninstall -y requests urllib3 chardet idna beautifulsoup4 tqdm six
#!pip install six
#!pip install requests
# You will need scikit-bio:
!pip install scikit-bio
from skbio.stats.ordination import cca

# Plot style
plt.rcParams["figure.figsize"] = (7, 5)
plt.rcParams["axes.grid"] = True

FIG_DIR = Path("figures")
FIG_DIR.mkdir(exist_ok=True)

Load dune data from R-universe

In [ ]:
def load_dune_data():
   # Load dune species and environmental data by embedding it directly.

   # Returns

   # dune : pd.DataFrame
   #     Species abundance table (sites x species).
   # dune_env : pd.DataFrame
   #     Environmental variables (sites x env variables).

   # Ensure expected columns
   # (dune: 30 species; dune_env: A1, Moisture, Management, Use, Manure)
    import rpy2.robjects as ro
    from rpy2.robjects.packages import importr
    from rpy2.robjects import pandas2ri

# Import the R 'utils' package for certain functions (like data() or install.packages())
    utils = importr('utils')

# Import the R package containing the 'dune' data (e.g., 'vegan')
# rpy2 automatically translates R's dot notation to Python's underscore notation
# but the package name itself is imported directly
    utils.install_packages('vegan') # Ensure vegan is installed

# Load the 'dune' and 'dune.env' datasets explicitly from the 'vegan' package
    ro.r('data("dune", package="vegan")')
    ro.r('data("dune.env", package="vegan")')

# Access the loaded R data frame from Python's global environment
    dune_r_dataframe = ro.r['dune']
    dune_env_r_dataframe = ro.r['dune.env']

# Convert R data frame to a pandas DataFrame
    pandas2ri.activate()
    dune_pandas_df = pandas2ri.rpy2py(dune_r_dataframe)
    dune_env_pandas_df = pandas2ri.rpy2py(dune_env_r_dataframe)

    return dune_pandas_df, dune_env_pandas_df


dune, dune_env = load_dune_data()
display(dune.head())
display(dune_env.head())

Derive richness and Shannon diversity, build MR dataset

In [ ]:
def compute_diversity_metrics(dune_df: pd.DataFrame) -> pd.DataFrame:

    #Compute species richness and Shannon diversity for each site.

    #Parameters
    #----------
    #dune_df : DataFrame
    #    Species abundance / cover matrix (sites x species).

    #Returns
    #-------
    #metrics : DataFrame
    #    Columns: 'richness', 'shannon'.

    # Convert cover classes to numeric counts (assume >0 = present)
    abundance = dune_df.astype(float).clip(lower=0.0)

    # Richness: number of species with positive abundance
    richness = (abundance > 0).sum(axis=1)

    # Shannon: H' = -sum p * log p
    row_sums = abundance.sum(axis=1)
    # Avoid /0 by dropping all-zero rows (shouldn't exist here)
    valid = row_sums > 0
    p = abundance[valid].div(row_sums[valid], axis=0)
    shannon = pd.Series(0.0, index=abundance.index)
    shannon[valid] = -(p * np.log(p.replace(0, np.nan))).sum(axis=1).fillna(0.0)

    metrics = pd.DataFrame({
        "richness": richness,
        "shannon": shannon
    }, index=dune_df.index)

    return metrics


div_metrics = compute_diversity_metrics(dune)
display(div_metrics.head())

Correlation matrix and heatmap (saved for LaTeX Figure mr_corr_matrix)

In [ ]:
# For correlations, convert ordered factors to codes
env_for_corr = dune_env.copy()
for col in ["Moisture", "Use", "Manure", "Management"]:
    if col in env_for_corr.columns:
        env_for_corr[col] = env_for_corr[col].astype("category").cat.codes

corr_df = pd.concat([div_metrics, env_for_corr], axis=1)
corr = corr_df.corr()

plt.figure()
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation matrix: diversity metrics and environmental variables")
plt.tight_layout()
plt.savefig(FIG_DIR / "mr_corr_matrix.png", dpi=300)
plt.close()

Build multivariate regression + regularized models

In [ ]:
# Prepare MR design matrix X and response Y
X_env = dune_env.copy()
numeric_features = ["A1"]
categorical_features = ["Moisture", "Management", "Use", "Manure"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(drop="first", sparse_output=False), categorical_features),
    ]
)

Y = div_metrics[["richness", "shannon"]].values

# OLS multivariate regression
ols_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("reg", LinearRegression())
    ]
)

ols_pipeline.fit(X_env, Y)
Y_hat_ols = ols_pipeline.predict(X_env)

# Metrics per response
results_mr = {}
for i, name in enumerate(["richness", "shannon"]):
    y_true = Y[:, i]
    y_pred = Y_hat_ols[:, i]
    r2 = r2_score(y_true, y_pred)
    n, p = X_env.shape
    # crude adjusted R^2 using number of encoded cols
    X_trans = ols_pipeline.named_steps["preprocess"].transform(X_env)
    p_eff = X_trans.shape[1]
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p_eff - 1)
    mse = mean_squared_error(y_true, y_pred)
    results_mr[name] = {"R2": r2, "Adj_R2": adj_r2, "MSE": mse}

results_mr

Residual diagnostics (richness + Shannon) → PNGs

In [ ]:
def diagnostic_plots(y_true, y_pred, base_name: str):
    resid = y_true - y_pred

    # Q-Q plot
    from scipy import stats
    fig, ax = plt.subplots()
    stats.probplot(resid, dist="norm", plot=ax)
    ax.set_title(f"Q-Q plot of residuals ({base_name})")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"mr_resid_{base_name}_qq.png", dpi=300)
    plt.close()

    # Residuals vs fitted
    fig, ax = plt.subplots()
    ax.scatter(y_pred, resid)
    ax.axhline(0, color="black", linewidth=1)
    ax.set_xlabel("Fitted values")
    ax.set_ylabel("Residuals")
    ax.set_title(f"Residuals vs fitted ({base_name})")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"mr_resid_{base_name}_fitted.png", dpi=300)
    plt.close()


diagnostic_plots(Y[:, 0], Y_hat_ols[:, 0], base_name="richness")
diagnostic_plots(Y[:, 1], Y_hat_ols[:, 1], base_name="shannon")

Ridge and multitask lasso, coefficient comparison plot

In [ ]:
# Transformed design once (pipeline handles it but we need feature names)
preprocess.fit(X_env)
X_trans = preprocess.transform(X_env)

# Feature names from the preprocessor
num_names = numeric_features
cat_names = list(
    preprocess.named_transformers_["cat"].get_feature_names_out(categorical_features)
)
feature_names = num_names + cat_names

# Ridge (multi-output)
alphas = np.logspace(-3, 3, 7)
ridge = RidgeCV(alphas=alphas)
ridge.fit(X_trans, Y)
B_ridge = ridge.coef_  # shape (n_targets, n_features)

# Multitask lasso (joint sparsity over responses)
lasso = MultiTaskLassoCV(
    alphas=np.logspace(-3, 1, 20),
    cv=min(5, len(X_trans)),
    random_state=42
)
lasso.fit(X_trans, Y)
B_lasso = lasso.coef_  # shape (n_targets, n_features)

# OLS coefficients in transformed space
ols = ols_pipeline.named_steps["reg"]
B_ols = ols.coef_  # (n_targets, n_features)

# Build tidy coefficient table
coef_df = pd.DataFrame({
    "feature": feature_names,
    "OLS_richness": B_ols[0, :],
    "OLS_shannon": B_ols[1, :],
    "Ridge_richness": B_ridge[0, :],
    "Ridge_shannon": B_ridge[1, :],
    "Lasso_richness": B_lasso[0, :],
    "Lasso_shannon": B_lasso[1, :],
})

coef_long = coef_df.melt(
    id_vars="feature",
    var_name="model_response",
    value_name="coef"
)

plt.figure(figsize=(10, 6))
sns.barplot(
    data=coef_long,
    x="feature",
    y="coef",
    hue="model_response"
)
plt.xticks(rotation=90)
plt.title("Coefficient comparison: OLS, Ridge, Multitask Lasso")
plt.tight_layout()
plt.savefig(FIG_DIR / "mr_coeffs_comparison.png", dpi=300)
plt.close()

CCA with scikit-bio

In [ ]:
def run_cca(dune_df: pd.DataFrame, dune_env_df: pd.DataFrame):
    """
    Run canonical correspondence analysis using scikit-bio.

    Returns
    -------
    ord_res : OrdinationResults
    species_scores : DataFrame
    site_scores : DataFrame
    env_scores : DataFrame
    """
    # Species matrix: fill NaNs with 0 as cca cannot handle them
    Y_cca = dune_df.fillna(0).copy()

    # Before filtering, identify and report rows with only zeros
    rows_all_zeros_before_filter = (Y_cca.sum(axis=1) == 0)
    print(f"Number of rows with all zeros in Y_cca before filtering: {rows_all_zeros_before_filter.sum()}")

    # Remove rows (sites) with only 0's from species data and corresponding environmental data
    non_empty_sites = Y_cca.sum(axis=1) > 0
    Y_cca = Y_cca[non_empty_sites]
    dune_env_df_filtered = dune_env_df[non_empty_sites]

    # After filtering, verify no rows have only zeros
    rows_all_zeros_after_filter = (Y_cca.sum(axis=1) == 0)
    if rows_all_zeros_after_filter.any():
        print("WARNING: Y_cca still contains rows with all zeros after filtering!")
        print(Y_cca[rows_all_zeros_after_filter])
    else:
        print("Y_cca successfully filtered; no rows with all zeros found.")

    # Further filter out species (columns) with all zeros from Y_cca
    non_empty_species = Y_cca.sum(axis=0) > 0
    Y_cca = Y_cca.loc[:, non_empty_species]
    print(f"Number of species with all zeros removed: {(~non_empty_species).sum()}")
    print(f"Y_cca shape after species filtering: {Y_cca.shape}")

    # Environmental matrix: dummy-coded, use filtered env data
    X_cca = pd.get_dummies(dune_env_df_filtered, drop_first=True)

    ord_res = cca(Y_cca, X_cca)
    species_scores = ord_res.features.iloc[:, :2]
    site_scores = ord_res.samples.iloc[:, :2]
    env_scores = ord_res.biplot_scores.iloc[:, :2]

    return ord_res, species_scores, site_scores, env_scores, X_cca.columns


ord_res, species_scores, site_scores, env_scores, env_names = run_cca(dune, dune_env)
ord_res

CCA eigenvalues plot

In [ ]:
eigvals = ord_res.eigvals
prop = eigvals / eigvals.sum()
cumprop = prop.cumsum()

fig, ax1 = plt.subplots()
ax1.bar(range(1, len(eigvals) + 1), eigvals, label="Eigenvalues")
ax1.set_xlabel("CCA axis")
ax1.set_ylabel("Eigenvalue")
ax2 = ax1.twinx()
ax2.plot(range(1, len(eigvals) + 1), cumprop, marker="o", color="tab:red", label="Cumulative proportion")
ax2.set_ylabel("Cumulative proportion of constrained inertia")

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="upper right")

plt.title("CCA eigenvalues and cumulative constrained inertia")
plt.tight_layout()
plt.savefig(FIG_DIR / "cca_eigenvalues.png", dpi=300)
plt.close()

CCA biplots (sites + env; species)

In [ ]:
# Site scores colored by Management, with env vectors
cca_df_sites = site_scores.copy()

# Only proceed with site biplot if at least one CCA component is available
if len(site_scores.columns) >= 1:
    cca_df_sites["Management"] = dune_env["Management"].astype(str)

    fig, ax = plt.subplots(figsize=(7, 6))
    if len(site_scores.columns) >= 2:
        # Plot 2D biplot if at least two components are available
        for mtype, sub in cca_df_sites.groupby("Management"):
            ax.scatter(sub.iloc[:, 0], sub.iloc[:, 1], label=mtype)
        ax.set_xlabel("CCA1")
        ax.set_ylabel("CCA2")
        ax.set_title("CCA site scores (colored by management)")

        # Environmental vectors from origin if at least two components are available
        if len(env_scores.columns) >= 2:
            for name, (xv, yv) in env_scores.iterrows():
                ax.arrow(0, 0, xv, yv, head_width=0.05, length_includes_head=True, color="black")
                ax.text(xv * 1.05, yv * 1.05, name, fontsize=9)
        else:
            print("Warning: Not enough CCA components for 2D environmental vectors on site biplot.")

    else:
        # Plot 1D if only one component is available
        print("Warning: Only 1 CCA component available for site scores. Plotting 1D.")
        for mtype, sub in cca_df_sites.groupby("Management"):
            ax.scatter(sub.iloc[:, 0], np.zeros_like(sub.iloc[:, 0]), label=mtype)
        ax.set_xlabel("CCA1")
        ax.set_ylabel("") # No CCA2 axis
        ax.set_title("CCA site scores (colored by management - 1D)")

    ax.legend(title="Management")
    plt.tight_layout()
    plt.savefig(FIG_DIR / "cca_sites_biplot_management.png", dpi=300)
    plt.close()
else:
    print("Warning: No CCA components available for site scores. Skipping site biplot.")
    plt.close()


# Species scores
# Only proceed with species biplot if at least one CCA component is available
if len(species_scores.columns) >= 1:
    fig, ax = plt.subplots(figsize=(7, 6))
    if len(species_scores.columns) >= 2:
        # Plot 2D biplot if at least two components are available
        ax.scatter(species_scores.iloc[:, 0], species_scores.iloc[:, 1], s=20)
        for sp, (xs, ys) in species_scores.iterrows():
            ax.text(xs, ys, sp, fontsize=7)
        ax.set_xlabel("CCA1")
        ax.set_ylabel("CCA2")
        ax.set_title("CCA species scores")
    else:
        # Plot 1D if only one component is available
        print("Warning: Only 1 CCA component available for species scores. Plotting 1D.")
        ax.scatter(species_scores.iloc[:, 0], np.zeros_like(species_scores.iloc[:, 0]), s=20)
        for sp, xs in species_scores.iloc[:, 0].items():
            ax.text(xs, 0, sp, fontsize=7)
        ax.set_xlabel("CCA1")
        ax.set_ylabel("") # No CCA2 axis
        ax.set_title("CCA species scores (1D)")

    plt.tight_layout()
    plt.savefig(FIG_DIR / "cca_species_biplot.png", dpi=300)
    plt.close()
else:
    print("Warning: No CCA components available for species scores. Skipping species biplot.")
    plt.close()